# Processing of particle tracks 
### <font color='gray'>coded to work with experiments and simulation</font>

## _Import of python libraries used within the code_

### <font color ='gray'> load both code cells  </font>  

In [1]:
#HI HELLO
import numpy as np # fast array library
import pandas as pd # data frames library
# plot display type. use this to keep graphs within the notebook
import matplotlib
%matplotlib ipympl 

import matplotlib.pyplot as plt # plot library

import subprocess # these 2 lines below load OS functionality
import sys
import json

from IPython.display import display, HTML # screen on display in HTML library 

import sympy as sp # symbolic calculation library functions
from sympy import var # symbolic calculation library variables

import mpmath as mp # double (or higher) floating point precision library (not used)

import scipy.constants as ct # import mathematical constants
import scipy.stats as st
import scipy.special as spec

import ipywidgets as widgets # import 

from scipy.spatial import Voronoi, voronoi_plot_2d # import Voronoi functions

import scipy.optimize as optimize # optimazation library (not used)
from collections import Counter # (not used)
import time # timing functions (for computation time)

from scipy import signal # noise filtering functions

global iscaled 
iscaled = 0

global LX, LY
LX = 1000 -250
LY = 650-50

# system size for a given (npart, rho) configuration
def L(npart, rho):
    l = np.sqrt(npart*2*np.sqrt(3.)/rho)
    return l

# print progress function
# more elegant (python 3 alternative): print('\rhello', end='',flush=True)
def printp(string):
    sys.stdout.write('\r'+ str(string))
    sys.stdout.flush()
    
#from __future__ import print_function

# Text format for figures configuration (lines below)
from matplotlib import rc
rc('text', usetex=True)
plt.rcParams['font.size'] = 20
# special plotting functions
from matplotlib.collections import PolyCollection
import matplotlib.colors as mcolors

def set_up_graf(idf, LX, LY):
    global ax, fig_system, ss

    fig_system = plt.figure(idf, figsize=(6.5,6.5*LY/LX))
    ss=(72./fig_system.dpi)**2 # particle size
    ax = fig_system.add_subplot(1, 1, 1)
    ax.set_ylim([0,LY])
    ax.set_xlim([0,LX])
    ax.set_xlabel(r'$x/\sigma$',fontsize = 24)
    ax.set_ylabel(r'$y/\sigma$', fontsize = 24)


In [2]:
%matplotlib --list

Available matplotlib backends: ['tk', 'gtk', 'gtk3', 'wx', 'qt4', 'qt5', 'qt', 'osx', 'nbagg', 'notebook', 'agg', 'svg', 'pdf', 'ps', 'inline', 'ipympl', 'widget']


## _Read tracking data _ 

We explain the action of each of these functions below:

__get_info(hash_prefix) :__ Reads and prints info file with  _hash prefix_ hash code. See output table in <font color='red' >RUN CELL 0</font> below to grasp the info structure

__simple_pickle_read(hash_prefix) :__
This 'simple_pickle_read' takes a pkl.xz file _hash prefix_ hash code, and which contains xy positions and tracks (no velocities) and stores in 'tabla' pandas data frame.
   
  * input: hash_prefix
  * output: global variable _Ntracks_ (number of tracks) 

In [2]:
def pdisplay(info):
    display(HTML(info.to_html()))

# This function reads pickle binary file (.pkl) with trajectories
def get_info(series_directory, hash_prefix):
    global info
    nombre = '/data/Datos/Tracks/'+ series_directory +'/info/' + hash_prefix + '.txt'
    with open(nombre) as f:
        jsonstr = json.load(f)
    info = pd.json_normalize(jsonstr)
    pdisplay(info.T)
    return info
    
def simple_pickle_read(series_directory, hash_prefix):
    global Nframes, Ntracks
    # Read table in pickle format
    nombre = '/data/Datos/Tracks/' + series_directory + '/tracks/900fps/' + hash_prefix \
       + '.pkl.xz'
    tabla = pd.read_pickle(nombre, compression='infer')
    tabla = tabla.reset_index(drop=True) # por si los indices salen desordenados
    Nframes = np.max(tabla.frame)+1
    Ntracks = np.max(tabla.track)+1
    tabla = tabla.sort_values(by = ['frame', 'track']).reset_index(drop=True)
    return tabla

## _Reduce global table - functions_

__reset_track_indexes(tabla0):__ 
Eliminates 'lonely' tracks (particles tracked for just 1 frame), re-indexes so that no track indexes are empty
* input: _tabla0_ original data frame (of tracks)
* output: RETURNS clean _tabla_ data frame, without empty tracks; AND original _tabla0_ with empty tracks still included
* output structure: _tabla0_, _tabla_

__short_drop:(ishort, tabla):__ 
Eliminates tracks detected for n frames or less, re-indexes so that no track indexes are empty

* Input: _ishort_ (threshold length of tracks: shorter tracks are not kept), _tabla_ (original tracks data frame)

* Output: RETURNS _tabla_\__short_ (frame), that is the input table _tabla_ without tracks shorter than _ishort_


In [3]:
def reset_track_indexes(tabla0):
    global Ntracks, Nframes
    """ This function takes a dataframe in which some trajectory indexes
        are missing (maybe due to having deleted short trajectories) and
        resets indexes so that we can loop over the tracks with 'range' """
    # 'real_number_of_tracks' should be <=  than 'current_last_particle_index'
    tabla = tabla0.copy()
    Ntracks = len(set(tabla.track))
    original_indexes = np.sort(list(set(tabla.track)))
    unsort_indexes = original_indexes
    fixed_indexes = np.arange(0, Ntracks, step=1, dtype=int)
    if  (original_indexes == fixed_indexes).all()==False: # fix only if there are empty tracks
    # With these two lists we create a dictionary and map old values to new ones
        n_empty = np.max(tabla.track) - Ntracks
        replacement_dict = dict(zip(original_indexes, fixed_indexes))
        tabla.track = tabla.track.map(dict(zip(original_indexes, fixed_indexes)))
        print('no. of empty track indexes discarded: ', n_empty, '\n')
    else:
        print('nothing to fix\n')
    Ntracks = np.max(tabla.track)+1
    Nframes = np.max(tabla.frame)+1
    return tabla0, tabla

# INPUT
# ishort: number of minimum frames in a track (eliminates tracks under ishort time length)
# tabla: pandas Data Frame to shorten
# OUTPUT
def short_drop(ishort, tabla):
    global shorts_list, Ntracks
    shorts_list =[]
    Ntracks = np.max(tabla.track)+1
    for i in range(Ntracks):
        t1 = track(i,tabla,False)
        if  len(t1) < ishort+1:
            shorts_list.append(i)
            len0 = len(tabla)
            tabla = tabla.drop(t1['index'])
            texto = 'dropped track no. '+str(i)+'; data table length decreased in '+str(len(tabla)-len0)
            printp(texto)
    print('\n')
    tabla_short = tabla.sort_values(by = ['frame', 'track']).reset_index(drop=True)
    Nshorts = np.max(tabla_short.track)+1
    printp('Dropped out ' + str(Ntracks-Nshorts)+' short tracks out of ' + str(Ntracks))
    # the line above is necessary so that eliminates index voids and shuffling after short drop
    print('\n')
    return tabla_short


## _Get tracks and states from global table - functions_

__track(t_id, tabla, dropit) :__  builds track for one particle from the appropriate chunk of the source table, with only the lines for particle _t_\__id_ 

* Input: _t_\__id_ indice de track, _tabla_ source frame fuente (contains tracks), _dropit_ boolean; if =True then erases old index column (tracks do not always begin in frame 0); most of the time you just want _dropit_=True

* Output: RETURNS a chunk of source table _tabla_ with only the lines for particle _t_\__id_

__all\___ __tracks(tabla, dropit):__ repeats the process in _track_ function for all existing particles

* Input: _tabla_ (frame) data source, _dropit_ boolean variable set True to erase original table original line number

* Output: RETURNS _track_ array of frames each wiith one particle track; builds _tr_\__lengths_ array of tracks lengths


__state(it, tabla):__ gets one instantaneous state frome source frame _tabla_, at frame _it_

* Input: _it_ frame no. to get the state from; _tabla_ source frame data

* Output: RETURNS st frame chunk from _tabla_ source frame

__all__\___states(tabla):__ Builds all instantaneous states from the movie

* Input: _tabla_ source frame data

* Output: RETURNS array of frame chunks from _tabla_ source frame. each chunk being an instantaneous state

In [4]:
#####  INDIVIDUAL TRACKS ##############
# build a 't_id' indivitual track 
def trackf(t_id, tabla, dropit):
    t1 = tabla.loc[tabla.track == t_id].reset_index(drop=dropit)
    return t1

# build individual tracks from all kept tracks
# OUTPUT
# tr_lengths[i]: length of track no.  'i'. The total no. of tracks is stored in 'Ntracks'
def all_tracks(tabla,dropit):
    # length of track
    global tr_lengths
    tr_lengths = np.empty(Ntracks,dtype=int)
    tracks = [[] for i in range(Ntracks)]
    for i in range(Ntracks):
        tracks[i] = trackf(i,tabla,dropit)
        tr_lengths[i] = int(len(tracks[i]))
    return tracks

#####  INSTANTANEOUS STATES  ##############
# BUILD INSTANTANEOUS STATES OF THE SYSTEM
def state(it, tabla):
    st = tabla.loc[tabla.frame == it].reset_index(drop=True)
    # reset row index 
    #(otherwise keeps chunked index of the original table)
    return st

# build instantaneous states over all frames
def all_states(tabla):
    sts = [[] for i in range(Nframes)]
    for i in range(Nframes):
        sts[i] = state(i,tabla)
    return sts


## _Low-Pass filters_ 

__butter_lowpass(step,fps, arr) :__ Applies Butterworth low-pass filter to _arr_ array, with averaging width _step_, for a movie with _fps_ frame rate; _fr_ is the fraction of the maximum frequency that is allowed to pass

* Input: _fr_, _step_, _fps_, _arr_; as described above
* Output: RETURNS an array of the same size as _arr_

__cheby1_lowpass(step,fps, arr) :__ Applies Chebyshev type I low-pass filter to _arr_ array, with averaging width _step_, for a movie with _fps_ frame rate

* Input: _step_, _fps_, _arr_; as described above
* Output: RETURNS an array of the same size as _arr_



In [5]:
def butter_lowpass(fr,step,fps, arr):
    
    #N, Wn = signal.buttord(1./(1.*fps/step),1./fps, 1/step, fps*0.5)
    N, Wn = signal.buttord(fr/step, fr, 1/step, fps*0.5 ,0.5/fps)
    b, a = signal.butter(N, Wn,'low')
    yy = signal.filtfilt(b, a, np.squeeze(arr), padtype=None)
    return yy

def filter_tracks_butter(fr,step,fps,tabla):
    tabla_unfiltered = tabla.copy()
    for i in range(Ntracks):
        printp('Filtering positions for track no. ' + str(i+1) + ' of ' + str(Ntracks))
        xbb = np.array(tabla.loc[tabla['track']==i,'x'])
        xb = butter_lowpass(fr,step,info.fps[0], xbb)
        tabla.loc[tabla['track']==i,'x'] = xb
        ybb = tabla.loc[tabla['track']==i,'y']
        yb = butter_lowpass(fr,step,info.fps[0], ybb)
        tabla.loc[tabla['track']==i,'y'] = yb
    print('\n')
    return tabla_unfiltered, tabla

#def cheby1_lowpass(step,fps,arr):
#    N, Wn = signal.cheb1ord(1./(1.*fps/step),1./fps, step, fps*0.5)
#    b, a = signal.cheby1(N, 1./fps, Wn, 'low')
#    y = signal.filtfilt(b, a, arr)
#    return y


## _Get velocities (differences) and accelerations (differences of differences) from individual tracks_

__vels(tabla) :__ Ouputs velocities from tracks table; by decomposing into individual trajectories first

* Input: _tabla_ frame data source; str01: name of _x_ coordinate inpuy variable; str02: same for _y_ coordinate

* Output: RETURNS _tabla_ with 2 new columns (_tabla['vx']_, _tabla['vy']_)

In [6]:
def diffs(str01,str02,str1,str2,tabla):
    global last_id
    tabla[str1] = np.zeros(len(tabla)) # creates new columns for differences
    tabla[str2] = np.zeros(len(tabla))
    last_id = np.zeros(Ntracks, dtype=int) # creates array for storing strack last index
    for k in range(Ntracks):
        printp('diffs: ' + str(k+1)+ ' of ' + str(Ntracks) + ' tracks')
        t1 = track(k,tabla,False)
        tabla.loc[t1['index'][:-1],str1] = np.diff(t1[str01]) #store differences in new columns
        tabla.loc[t1['index'][:-1],str2] = np.diff(t1[str02])
        last_id[k] = tabla.loc[tabla['track']==k].index.values[-1:]
    tabla = tabla.drop(last_id) # gets rid of last point in track (has no defined diff)
    print('\n')
    return tabla
    
def angle_diffs(tabla):
    global last_id
    tabla['w'] = np.zeros(len(tabla)) # creates spin column
    for k in range(Ntracks):
        printp('diffs: ' + str(k+1)+ ' of ' + str(Ntracks) + ' tracks')
        t1 = track(k,tabla,False)
        tabla.loc[t1['index'][:-1],'w'] = np.diff(t1['theta']) 
        last_id[k] = tabla.loc[tabla['track']==k].index.values[-1:]
    tabla = tabla.drop(last_id) # gets rid of last point in track (has no defined diff)
    print('\n')
    return tabla

## Bring data to physical units

We take as coordinate origin the mid-point of the max and min X and Y positions ever tracked within the region of interest (ROI) during the experiment.

We have 2 options for lengh unit: 
* milimeters 
* Ball diameter

The time scale unit is seconds.

Only after this step position differences in the original data table are re-scaled as real velocities

In [7]:
# PHYSICAL SCALES AND ORIGIN
# Usage: re_pos_scale(0) for 1 mm as length unit; re_pos_scale(1) for ball diameter (sigma) 
# length unit

def set_origin(shiftx, shifty, tabla):
    global ishifted
    if ishifted == 0:
        tabla_not_shifted = tabla.copy()
        tabla['x'] -= shiftx
        tabla['y'] -= shifty
        shifted = 1
    return tabla_not_shifted, tabla


def scale(l_unit, t_unit, tabla):
    global iscaled
    tabla_not_scaled = tabla.copy()
    if iscaled == 0:  
        inv_l_unit = 1./l_unit
        print(inv_l_unit)
        if np.any(tabla.columns=='vx') or np.any(tabla.columns=='vy'):
            print(inv_l_unit)
            tabla[['x','y']] *= inv_l_unit
            tabla[['vx', 'vy']] *= (inv_l_unit * t_unit)
            if np.any(tabla.columns=='ax') or np.any(tabla.columns=='ay'):
                tabla[['ax', 'ay']] *=  (inv_l_unit * t_unit**2)
        else:
            tabla[['x','y']] *= inv_l_unit
        iscaled = 1
    else:
        tabla[['x','y']] *= 1
        iscaled = 0
    return tabla_not_scaled, tabla


## <font color='RED'>RUN CELL 0</font>
### - Import pkl.xz tracks table
### - create all individual tracks arrays
### - analyze track length histogram, decide minimum track length

In [8]:
# COMPLETE SET OF READING INSTRUCTIONS

series_directory = 'chiral/rho_0_025'
hash_prefix = '4bf1a08dbd71379766e543b9532df560'

info = get_info(series_directory, hash_prefix)

datos = simple_pickle_read(series_directory, hash_prefix)
datos_orig, datos = reset_track_indexes(datos)

datos = datos[['frame','track','x','y','extremos']]

maxs = np.array([datos.extremos[i][0] for i in range(len(datos))])
#mins = np.array([datos.extremos[i][1] for i in range(len(datos))])

del datos['extremos']

maxs = maxs/180
#mins = mins/180

datos['maxt'] = maxs
#datos['mint'] = mins


for i in range(len(datos)):
    datos.at[i,'maxt'] = np.array(list(filter(lambda x: x<2, datos.maxt[i])))

datos['cuad'] = datos['maxt']

#datos.columns = ['frame','track','x','y','maxs','mins'];

# imprime (formateada) cabecera de tabla de datos
display(HTML(datos.head().to_html()))

print('\n min for positions:\n', np.min(datos[['x','y']]), '\n')
print('max. for positions:\n', np.max(datos[['x','y']]), '\n')

## store tracks individually
tracks = all_tracks(datos, True)

,0
experiment_id,4bf1a08dbd71379766e543b9532df560
original_file,/mnt/beegfs/malopez/serieAspas/serieAspas_N25_p20_fps900_1.cine
date,2038-01-19 04:18:16
shape,"[1280, 800]"
fps,900
exposure,1109
n_frames,24981
recording_time,27.7567
camera_distance,0.95
pixel_ratio,1089


nothing to fix



,frame,track,x,y,maxt,cuad
0,1,0,814.647468,643.994447,"[0.036626783701913065, 0.18385052658094192, 0.3383244591267481, 0.4562430450572031, 0.5804050080754658, 0.7183706507015212, 0.8644398598688607, 0.9999052086191216, 1.1503496663110337, 1.2929713542490024, 1.4363411941299624, 1.5874308181968726, 1.7412504103398132, 1.8881570675411483]","[0.036626783701913065, 0.18385052658094192, 0.3383244591267481, 0.4562430450572031, 0.5804050080754658, 0.7183706507015212, 0.8644398598688607, 0.9999052086191216, 1.1503496663110337, 1.2929713542490024, 1.4363411941299624, 1.5874308181968726, 1.7412504103398132, 1.8881570675411483]"
1,1,1,833.440082,155.107584,"[0.1327300598204992, 0.27908924051532835, 0.4054216786116568, 0.5516948678541602, 0.689981271549557, 0.8421728254098394, 0.9827381382866865, 1.119900593440402, 1.259020667510297, 1.396324537786448, 1.5307304849910386, 1.6795844111219265, 1.8382726334788568, 1.9810270061086195]","[0.1327300598204992, 0.27908924051532835, 0.4054216786116568, 0.5516948678541602, 0.689981271549557, 0.8421728254098394, 0.9827381382866865, 1.119900593440402, 1.259020667510297, 1.396324537786448, 1.5307304849910386, 1.6795844111219265, 1.8382726334788568, 1.9810270061086195]"
2,1,2,712.325766,155.562226,"[0.08782313512565135, 0.2349385832673767, 0.38788240633323867, 0.5241541211354374, 0.6563221401950123, 0.8038474391643975, 0.9439634387320286, 1.0819570364850377, 1.2185599646744307, 1.344617381451658, 1.4940962714541342, 1.6381167121184386, 1.7875803763250517, 1.940001221635273]","[0.08782313512565135, 0.2349385832673767, 0.38788240633323867, 0.5241541211354374, 0.6563221401950123, 0.8038474391643975, 0.9439634387320286, 1.0819570364850377, 1.2185599646744307, 1.344617381451658, 1.4940962714541342, 1.6381167121184386, 1.7875803763250517, 1.940001221635273]"
3,1,3,655.528404,229.414274,"[0.16776375319657666, 0.31667411487024094, 0.43796018389235447, 0.5920296371063434, 0.7258939812005311, 0.8699922181456345, 1.0079765582699025, 1.1580646818469185, 1.2881824793268868, 1.428371324620072, 1.566344401990137, 1.727039962380502, 1.8722527741192538]","[0.16776375319657666, 0.31667411487024094, 0.43796018389235447, 0.5920296371063434, 0.7258939812005311, 0.8699922181456345, 1.0079765582699025, 1.1580646818469185, 1.2881824793268868, 1.428371324620072, 1.566344401990137, 1.727039962380502, 1.8722527741192538]"
4,1,4,574.409841,238.649058,"[0.04480840352667365, 0.19640929912144098, 0.3536709057984168, 0.47248630606323455, 0.6189222452045975, 0.7641760936411512, 0.9066594888435884, 1.0339342498555228, 1.1780735429491505, 1.307872820287969, 1.4453061184480467, 1.5927021972422895, 1.749193259130297, 1.8955330813656202]","[0.04480840352667365, 0.19640929912144098, 0.3536709057984168, 0.47248630606323455, 0.6189222452045975, 0.7641760936411512, 0.9066594888435884, 1.0339342498555228, 1.1780735429491505, 1.307872820287969, 1.4453061184480467, 1.5927021972422895, 1.749193259130297, 1.8955330813656202]"



 min for positions:
 x    437.795667
y    122.745921
dtype: float64 

max. for positions:
 x    986.920536
y    690.644348
dtype: float64 



In [12]:
theta0 = tracks[1].maxt[0][0]; print(theta0)

0.1327300598204992


In [48]:
j = 0
it = 0 
diff0 = tracks[1].maxt[it+1][j] - theta0 # ldiff0, rdiff0
if (j==13):
    ldiff1 = tracks[1].maxt[it+1][0] - theta0 + 2 # ldiffm2
else:
    ldiff1 = tracks[1].maxt[it+1][j+1] - theta0 # ldiff1
if (j==0):
    rdiff1 = -(tracks[1].maxt[it+1][13] - theta0 - 2) #rdiffM2
else:
    rdiff1 = tracks[1].maxt[it+1][j-1] - theta0
next0 = min(diff0,abs(ldiff1),rdiff1); print(diff0,rdiff1,ldiff1)
print(next0)

0.009158918934119076 0.13513680471039824 0.1584304883924
0.009158918934119076


In [132]:
nblades = 14
def return_next(itr,jt,j0):
    theta0 = tracks[itr].maxt[jt][j0]
    diff0 = tracks[itr].maxt[jt+1][j0] - theta0 # ldiff0, rdiff0
    extt = 0
    if (j0==nblades-1):
        ldiff1 = tracks[itr].maxt[jt+1][0] - theta0 + 2 # ldiffm2
        extt = 1
    else:
        ldiff1 = tracks[itr].maxt[jt+1][j0+1] - theta0 # ldiff1
        
    if (j0==0):
        rdiff1 = -(tracks[itr].maxt[jt+1][nblades-1] - theta0 - 2) #rdiffM2
        extt = 1
    else:
        rdiff1 = tracks[itr].maxt[jt+1][j0-1] - theta0 #rdiff1
    
    nm = min(diff0,abs(ldiff1),rdiff1)
    in0 = np.where(np.array([diff0,abs(ldiff1),rdiff1])== nm)[0][0] 
    #referir a lista [tracks[itr].maxt[jt+1][j0] - theta0, tracks[itr].maxt[jt+1][0] - theta0 + 2, ... ]
    #creo que así lo de abajo sobraría
    if (extt>0 and in0>0):
        next0 = np.array([0,nblades-1])[in0-1]
    else: 
        next0 = np.array([j0,j0+1,j0-1])[in0]
    print(nm)
    print(in0)
    print(extt)
    print(next0)
    #return np.where(tracks[itr].maxt[jt+1]+thetanm)

In [133]:
return_next(1,0,0)

0.009158918934119076
0
1
0


In [35]:
nblades = 14
def shift_back(arr,n0):
    arr_copy = np.zeros(nblades)
    for i in range(n0,nblades):
        arr_copy[i-n0] = arr[i]
    for i in range(0,n0):
        arr_copy[(nblades-1)-(n0-1)+i] = arr[i]
    return arr_copy

In [36]:
tracks[1].maxt[1]

array([0.14188898, 0.29116055, 0.43208482, 0.56274166, 0.71612866,
       0.85908847, 1.00266288, 1.13823514, 1.27034512, 1.40996041,
       1.54763743, 1.69813643, 1.84620388, 1.99759326])

In [38]:
shift_back(tracks[1].maxt[1],9)

array([1.40996041, 1.54763743, 1.69813643, 1.84620388, 1.99759326,
       0.14188898, 0.29116055, 0.43208482, 0.56274166, 0.71612866,
       0.85908847, 1.00266288, 1.13823514, 1.27034512])

In [19]:
for i in range(4,14):
    print(i)

4
5
6
7
8
9
10
11
12
13


In [21]:
for i in range(0,4):
    print(i)

0
1
2
3


In [13]:
len(tracks[1].maxt[1])

14

In [255]:
nm = np.min(tracks[1].maxt[1]-theta0)
nm2 = np.min(tracks[1].maxt[1]+2-theta0)
if (nm<nm2):
    next0 = np.where((nm +theta0 == tracks[1].maxt[1]) ==True)[0][0]
else:
     next0 = np.where((nm -2 + theta0 == tracks[1].maxt[1]) ==True)[0][0]
print(next0, nm)

0 0.009158918934119076


In [95]:
a = np.array([1, 2, 3])
np.where(a==3)[0][0].astype('int')


2

In [250]:
tracks[1].maxt[1]+2-theta0

array([2.00915892, 2.15843049, 2.29935476, 2.4300116 , 2.5833986 ,
       2.72635841, 2.86993282, 3.00550508, 3.13761506, 3.27723035,
       3.41490737, 3.56540637, 3.71347382, 3.8648632 ])

In [244]:
tracks[1].maxt[1]-theta0

array([0.00915892, 0.15843049, 0.29935476, 0.4300116 , 0.5833986 ,
       0.72635841, 0.86993282, 1.00550508, 1.13761506, 1.27723035,
       1.41490737, 1.56540637, 1.71347382, 1.8648632 ])

In [245]:
lnm = np.max(tracks[1].maxt[1]-2-theta0)
lnext0 = np.where((lnm + 2 + theta0 == tracks[1].maxt[1]) ==True)[0][0]
print(lnext0)

13


In [247]:
min(nm,np.abs(lnm))

0.009158918934119076

In [243]:
tracks[1].maxt[1]-2-theta0

array([-1.99084108, -1.84156951, -1.70064524, -1.5699884 , -1.4166014 ,
       -1.27364159, -1.13006718, -0.99449492, -0.86238494, -0.72276965,
       -0.58509263, -0.43459363, -0.28652618, -0.1351368 ])

In [207]:
n

array([False, False, False])

In [202]:
np.abs(tracks[1].maxt[1][-1]-2-theta0)

0.1351368047103982

In [146]:
## store tracks individually
#tracks = all_tracks(datos, True)
# Histogram of trajectory length
plt.figure(figsize=(8,8/ct.golden))
plt.title('Trajectory lengths')
# set upper height limit for histogram
track_lengths = [len(tracks[i]) for i in range(Ntracks)]
# number of bins for track length histogram
nbins = 200
#plot
plt.ylim(0,1.25*Ntracks)
fig=plt.hist( track_lengths, nbins, color='b')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [153]:
plt.figure(figsize=(8,8/ct.golden))

# Histogram of trajectory length
datos_length = len(datos.maxt)-1


# set upper height limit for histogram
edges_lengths = [len(datos.maxt[i]) for i in range(datos_length)]
# number of bins for track length histogram


fig = plt.hist( edges_lengths, bins =[11,12,13,14,15, 16],\
                color='b',density=True, align='left' ,rwidth=0.95)

s= '11 edges: ' + str("{0:.2%}".format(fig[0][0])) +\
'\n12 edges: ' + str("{0:.2%}".format(fig[0][1])) +\
'\n13 edges: ' + str("{0:.2%}".format(fig[0][2])) +\
'\n14 edges: ' + str("{0:.2%}".format(fig[0][3])) + \
'\n15 edges: ' + str("{0:.4%}".format(fig[0][4]))

plt.text(11.75, 0.7, s= s , horizontalalignment='center',fontsize=12)

plt.title('propellers no. of edges')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'propellers no. of edges')

In [108]:
angulos = np.array([datos.maxt[i][0] for i in range(len(datos))])

In [109]:
plt.figure(figsize=(8,8/ct.golden))
plt.hist( angulos*(14/2)%1, bins=10,\
                color='b',density=True, align='left' ,rwidth=0.95)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([1.18317491, 1.26867109, 0.91781487, 0.96526293, 0.93129826,
        0.96340536, 0.93831218, 0.9645103 , 0.93281954, 0.93477319]),
 array([3.39595906e-06, 1.00002970e-01, 2.00002543e-01, 3.00002117e-01,
        4.00001691e-01, 5.00001264e-01, 6.00000838e-01, 7.00000412e-01,
        7.99999986e-01, 8.99999559e-01, 9.99999133e-01]),
 <a list of 10 Patch objects>)

In [138]:
for i in range(len(datos)):
    datos.at[i,'maxt'] = np.array(list(filter(lambda x: x<2, datos.maxt[i])))

In [139]:
tracks = all_tracks(datos, True)

## <font color='RED'>RUN CELL 1</font>

### - obtain position differences  (vels)
### - obtain velocities differences  (accels)
### - create all (non-short) individual tracks arrays. (minimum theoretical length: 5)

In [243]:

## scale and re-position the system (left-bottom corner is (0,0))
ishifted = 0

not_shifted, datos = \
set_origin(info.ROI_center[0][0],info.ROI_center[0][1], datos) # re-position

iscaled = 0
not_scaled, datos = \
scale(info.particle_diameter_px[0], 10**3/info.fps[0], datos) # scale

#
## calculate velocities for all tracks
#datos = diffs('x','y','vx','vy', datos);
#
##calculate accelerations for all tracks
#tracks_table[ishort] = diffs('vx','vy','ax','ay', datos);
#print table head
#
#print('scaled: \n')
display(HTML( datos.head().to_html() ))
#
#
## store tracks individually
#tracks = all_tracks(datos, True)
#
## store system (instantaneous) states individually
#states = all_states(datos)


0.012658227848101266


,frame,track,x,y,theta
0,1,0,-8.278377,-4.960103,"[[17.35402463626132, 45.0, 74.47588900324574, 98.13010235415598, 127.56859202882748, 154.79887635452494, 180.0, 205.20112364547506, 232.4314079711725, 257.4711922908485, 276.7098368077569, 298.07248693585296, 326.3099324740202, 353.99099404250546, 377.35402463626133, 405.0], [6.009005957494525, 32.005383208083494, 57.9946167919165, 83.6598082540901, 109.6538240580533, 137.48955292199918, 165.06858282186246, 190.0079798014413, 215.53767779197437, 241.69924423399365, 266.18592516570965, 286.3895403340348, 315.0, 341.565051177078, 366.00900595749454, 392.0053832080835, 417.9946167919165]]"
1,1,1,-8.275366,-5.038438,"[[3.1798301198642345, 29.74488129694222, 59.03624346792648, 83.99099404250548, 107.35402463626133, 132.51044707800085, 158.19859051364818, 180.0, 206.565051177078, 235.30484646876602, 262.40535663140855, 282.5288077091515, 310.23635830927384, 338.1985905136482, 363.1798301198642, 389.7448812969422], [18.43494882292201, 45.0, 71.56505117707799, 96.00900595749452, 116.56505117707799, 141.84277341263095, 167.47119229084848, 190.61965527615513, 216.86989764584402, 244.79887635452496, 270.0, 298.07248693585296, 327.2647737278924, 356.18592516570965, 378.434948822922, 405.0]]"
2,1,2,-8.294772,-5.038365,"[[9.462322208025617, 37.568592028827496, 66.37062226934319, 90.0, 116.56505117707799, 141.34019174590992, 166.75948008481282, 188.9726266148964, 214.69515353123396, 242.10272896905238, 270.0, 288.434948822922, 317.29061004263855, 345.06858282186244, 369.4623222080256, 397.5685920288275], [25.016893478100023, 50.71059313749964, 78.69006752597979, 101.30993247402021, 126.2538377374448, 151.69924423399362, 176.18592516570965, 199.44003482817618, 225.0, 251.565051177078, 276.3401917459099, 303.6900675259798, 331.92751306414704, 360.0, 385.01689347810003, 410.71059313749964]]"
3,1,3,-8.303873,-5.026532,"[[22.380135051959574, 49.398705354995535, 79.38034472384487, 101.30993247402021, 127.56859202882748, 151.92751306414706, 180.0, 202.38013505195957, 229.3987053549955, 254.47588900324575, 277.59464336859145, 302.7352262721076, 330.6422464572087, 360.0, 382.3801350519596, 409.3987053549955], [10.619655276155134, 37.568592028827496, 63.43494882292201, 90.0, 111.25050550713325, 137.29061004263855, 162.6459753637387, 186.3401917459099, 211.60750224624888, 237.9946167919165, 263.6598082540901, 288.434948822922, 317.4895529219991, 345.9637565320735, 370.61965527615513, 397.5685920288275]]"
4,1,4,-8.316871,-5.025052,"[[17.35402463626132, 45.0, 74.47588900324574, 97.59464336859145, 124.69515353123397, 149.03624346792648, 173.6598082540901, 195.94539590092285, 222.51044707800082, 249.44395478041653, 270.0, 295.20112364547504, 324.4623222080256, 352.8749836510982, 377.35402463626133, 405.0], [6.009005957494525, 32.7352262721076, 59.03624346792648, 83.99099404250548, 109.4400348281762, 132.87890360333856, 160.3461759419467, 180.0, 206.565051177078, 232.4314079711725, 259.38034472384487, 282.5288077091515, 310.6012946450045, 340.3461759419467, 366.00900595749454, 392.7352262721076]]"


## _Plotting functions_ 

__frameit(ax):__ frames a figure by drawing the image limits and the ROI limits as well

__plt__\___track(t_id,xs,tagit,fr\_it):__ plots just one track (_t_\__id_) within _fr_\__it_ (if True) frames, with size _xs_ and prints the track no. if _tagit_ is True

__plt__\___tracks(init_id, final_id,xs,tagit,fr\_it):__ plots _init_\__id_ to _final_\__id_ tracks within _fr_\__it_ (if True) frames, with size _xs_ and prints the track no. if _tagit_ is True

In [50]:
def frameit(ax):
    ax.set_xlim(0,info['shape'][0][0])
    ax.set_ylim(0,info['shape'][0][1])
    rect = plt.Rectangle([250,50], 750, 600, alpha=1, lw=10,fill=False, edgecolor='b')
    ax.add_artist(rect)

# Plot just one track function (tagit?, frameit?)
def plt_track(t_id,xs,tagit,fr_it):
    if fr_it==True:
        fig, ax = plt.subplots(figsize=(xs,xs*info['shape'][0][1]/info['shape'][0][0]))
        px_size = 72./fig.dpi
        frameit(ax)
    else:
        fig, ax = plt.subplots()
        px_size = 72./fig.dpi
        
    plt.plot(tracks[t_id].x,tracks[t_id].y, '.', c='r', markersize=px_size,linewidth=None)
    if tagit==True:
        plt.text(np.mean(tracks[t_id].x), np.mean(tracks[t_id].y), str(t_id))
            
def plt_tracks(init_id,final_id,xs,tagit,fr_it):
    if fr_it==True:
        fig, ax = plt.subplots(figsize=(xs,xs*info['shape'][0][1]/info['shape'][0][0]))
        px_size = 72./fig.dpi
        frameit(ax)
    else:
        fig, ax = plt.subplots()
        px_size = 72./fig.dpi
    for i in range(init_id,final_id):
        plt.plot(tracks[i].x,tracks[i].y, '.', c='r', markersize=px_size,linewidth=None)
        if tagit==True:
            plt.text(np.mean(tracks[i].x), np.mean(tracks[i].y), str(i))
